In [ ]:
import cv2, numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## Load Parameters

In [ ]:
from utils import read_json

params = read_json('parameters.json')

RESIZE_X = params['resize']['x']
RESIZE_Y = params['resize']['y']
ITEM_FOLDER = params['item_folder']

## Input Data
* Image of the bin
* List of items in the bin

In [ ]:
bin_stamp = '170405145336'
contents = ["Colgate_Toothbrush_4PK","Epsom_Salts","Duct_Tape",
            "Bath_Sponge","Crayons","Burts_Bees_Baby_Wipes"]

In [ ]:
bin_stamp = '170405145538'
contents = ["glue_sticks","tissue_box","laugh_out_loud_jokes",
            "toilet_brush","expo_eraser","table_cloth"]

In [ ]:
contents = [s.lower() for s in contents]

## Phase 1: Recognition by SIFT Features
### Compute Features in Bin

In [ ]:
from utils import imread_rgb, compute_sift

filename_bin = 'bin/' + bin_stamp + '.png'
image_bin = imread_rgb(filename_bin)
(kp_bin, des_bin) = compute_sift(image_bin)

### Match Bin and Items Features

In [ ]:
from utils import match_items

items = list(contents)
item_d, recognised_items, mask_items = match_items(image_bin, kp_bin, des_bin, items)

### Not recognised items

In [ ]:
items = [s for s in contents if s not in recognised_items]
items

In [ ]:
kernel = np.ones((3,3),np.uint8)
mask_items = cv2.dilate(mask_items,kernel,iterations = 5)
plt.imshow(mask_items,cmap='gray'), plt.axis('off');

## Phase 2: segment bin by depth into upper / lower levels

In [ ]:
from utils import imread_gray
filename_bin = 'bin/' + bin_stamp + '.pgm'
image_depth = imread_gray(filename_bin)
plt.imshow(image_depth,cmap='gray'); plt.axis('off');

In [ ]:
from utils import fill_holes

image_depth = fill_holes(image_depth)
plt.imshow(image_depth,cmap='gray');

In [ ]:
image_depth = cv2.bitwise_and(image_depth,image_depth,mask=255-mask_items)
pix_depth = image_depth.flatten()
pix_depth = pix_depth[pix_depth>0]
dmin, dmax = (np.min(pix_depth),np.max(pix_depth))
h,b = np.histogram(pix_depth,bins=dmax-dmin)
sh = np.cumsum(h)
b_high = [b_val for b_val,sh_val in zip(b,sh) if sh_val>sh[-1]/2]
dcut = int(b_high[0])
plt.hist(pix_depth,bins=dmax-dmin+1);
hmax = np.around(max(h)+1000,-3)
plt.plot([dcut,dcut],[0,hmax],'r-')
plt.axis([dmin,dmax,0,hmax]);

In [ ]:
upper_bin = cv2.inRange(image_depth,int(dmin),dcut-1)
kernel = np.ones((3,3),np.uint8)
upper_bin = cv2.dilate(upper_bin,kernel,iterations = 3)
plt.imshow(cv2.bitwise_and(image_bin,image_bin,mask=upper_bin));

In [ ]:
lower_bin = 255-upper_bin
#lower_bin = cv2.inRange(image_depth,dcut,int(dmax))
#kernel = np.ones((3,3),np.uint8)
#lower_bin = cv2.dilate(lower_bin,kernel,iterations = 3)
plt.imshow(cv2.bitwise_and(image_bin,image_bin,mask=lower_bin));

## Cluster colors in upper bin

In [ ]:
from utils import cluster_colors

positions, weights = cluster_colors(image_bin, upper_bin, items)

In [ ]:
pos_ok = [(p,w[0][1]) for p,w in zip(positions,weights) if len(w)==1]
pos_unkw = [(p,w) for p,w in zip(positions,weights) if len(w)>1]
it_col = {'table_cloth':(255,0,255),'tissue_box':(0,255,255),'glue_sticks':(255,0,0),'toilet_brush':(255,255,0)}
#it_col = {'burts_bees_baby_wipes':(255,0,255),'bath_sponge':(0,255,255),'duct_tape':(255,0,0)}
image_disp = image_bin.copy()
mask_disp = upper_bin.copy()
for item in items:
    it_pos = [p for p,it in pos_ok if it==item]
    for cnt in it_pos:
        cv2.drawContours(image_disp,cnt,-1,it_col[item],2)
        cv2.drawContours(mask_disp,cnt,-1,(255,),-1)
for pos,w in pos_unkw:
    for cnt in pos:
        cv2.drawContours(image_disp,cnt,-1,(0,0,0),1)
        #cv2.drawContours(mask_disp,cnt,-1,(255,),-1)
plt.imshow(image_disp); plt.axis('off');

In [ ]:
weights

## Cluster colors in lower bin

In [ ]:
positions, weights = cluster_colors(image_bin, lower_bin, items)

In [ ]:
pos_ok = [(p,w[0][1]) for p,w in zip(positions,weights) if len(w)==1]
pos_unkw = [(p,w) for p,w in zip(positions,weights) if len(w)>1]
it_col = {'table_cloth':(255,0,255),'tissue_box':(0,255,255),'glue_sticks':(255,0,0),'toilet_brush':(255,255,0)}
#it_col = {'burts_bees_baby_wipes':(255,0,255),'bath_sponge':(0,255,255),'duct_tape':(255,0,0)}
image_disp = image_bin.copy()
mask_disp = upper_bin.copy()
for item in items:
    it_pos = [p for p,it in pos_ok if it==item]
    for cnt in it_pos:
        cv2.drawContours(image_disp,cnt,-1,it_col[item],2)
        cv2.drawContours(mask_disp,cnt,-1,(255,),-1)
for pos,w in pos_unkw:
    for cnt in pos:
        cv2.drawContours(image_disp,cnt,-1,(0,0,0),1)
        #cv2.drawContours(mask_disp,cnt,-1,(255,),-1)
plt.imshow(image_disp); plt.axis('off');

In [ ]:
weights